In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc, confusion_matrix, accuracy_score
import pandas as pd
import os
from tensorflow.keras.models import load_model

def plot_metrics_row(model_path, X_test, y_test, history_path, save_path=None):
    """
    Plot ROC curve, training/validation accuracy, training/validation loss,
    and confusion matrix in a single row.

    Parameters:
    - model_path: path to the saved model
    - X_test: test features
    - y_test: test labels
    - history_path: path to training history CSV file
    - save_path: path to save the plot (optional)
    """

    # Load model and make predictions
    try:
        model = load_model(model_path)
        y_pred_prob = model.predict(X_test).flatten()
        y_pred = (y_pred_prob > 0.5).astype(int)
    except Exception as e:
        print(f"Error loading model: {e}")
        return

    # Load training history
    try:
        history_df = pd.read_csv(history_path)
    except Exception as e:
        print(f"Error loading history: {e}")
        return

    # Create figure with 1 row and 4 columns
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    fig.suptitle('Model Performance Metrics', fontsize=16, fontweight='bold')

    # 1. ROC Curve
    fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
    roc_auc = auc(fpr, tpr)

    axes[0].plot(fpr, tpr, color='darkorange', lw=2,
                label=f'ROC curve (AUC = {roc_auc:.3f})')
    axes[0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    axes[0].set_xlim([0.0, 1.0])
    axes[0].set_ylim([0.0, 1.05])
    axes[0].set_xlabel('False Positive Rate')
    axes[0].set_ylabel('True Positive Rate')
    axes[0].set_title('ROC Curve')
    axes[0].legend(loc="lower right")
    axes[0].grid(True, alpha=0.3)

    # 2. Training and Validation Accuracy
    epochs = range(1, len(history_df) + 1)
    axes[1].plot(epochs, history_df['accuracy'], 'b-', label='Training Accuracy', linewidth=2)
    axes[1].plot(epochs, history_df['val_accuracy'], 'r-', label='Validation Accuracy', linewidth=2)
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy')
    axes[1].set_title('Training & Validation Accuracy')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    # 3. Training and Validation Loss
    axes[2].plot(epochs, history_df['loss'], 'b-', label='Training Loss', linewidth=2)
    axes[2].plot(epochs, history_df['val_loss'], 'r-', label='Validation Loss', linewidth=2)
    axes[2].set_xlabel('Epoch')
    axes[2].set_ylabel('Loss')
    axes[2].set_title('Training & Validation Loss')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)

    # 4. Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Healthy', 'Parkinson'],
                yticklabels=['Healthy', 'Parkinson'],
                ax=axes[3])
    axes[3].set_xlabel('Predicted')
    axes[3].set_ylabel('Actual')
    axes[3].set_title('Confusion Matrix')

    # Add accuracy text to confusion matrix
    test_accuracy = accuracy_score(y_test, y_pred)
    axes[3].text(0.5, -0.1, f'Test Accuracy: {test_accuracy:.3f}',
                ha='center', va='center', transform=axes[3].transAxes,
                fontsize=10, fontweight='bold')

    # Adjust layout
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    # Save plot if path provided
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Plot saved to: {save_path}")

    plt.show()

def main():
    """
    Main function to run the plotting script.
    Update these paths according to your setup.
    """
    # Update these paths based on your configuration
    DATASET = "ITALIAN_DATASET"
    MODE = "A"
    FEATURE_MODE = "DEFAULT"

    # Determine dataset name
    if DATASET == "NEUROVOZ_DATASET":
        dataset = "Neurovoz"
    elif DATASET == "UAMS_DATASET":
        dataset = "UAMS"
    elif DATASET == "MPOWER_DATASET":
        dataset = "mPower"
    elif DATASET == "SYNTHETIC_DATASET":
        dataset = "Synthetic"
    elif DATASET == "ITALIAN_DATASET":
        dataset = "Italian"

    # Paths
    BASE_DIR = os.getcwd()
    FEATURES_FILE_PATH = os.path.join(BASE_DIR, dataset, "data", f"features_{MODE}_{FEATURE_MODE}.npz")
    MODEL_PATH = os.path.join(BASE_DIR, dataset, f"results_{MODE}_{FEATURE_MODE}", "cnn_att_lstm")
    BEST_MODEL_PATH = os.path.join(MODEL_PATH, "best_model.keras")
    HISTORY_PATH = os.path.join(MODEL_PATH, "history.csv")
    PLOT_SAVE_PATH = os.path.join(MODEL_PATH, "metrics_overview.png")

    # Load test data
    print("Loading test data...")
    try:
        with np.load(FEATURES_FILE_PATH) as data:
            labels = data['labels']
            mel_spectrogram = data['mel_spectrogram']
            mfcc = data['mfcc']
            X = np.concatenate((mel_spectrogram, mfcc), axis=-1)

        # Split data (use same random_state as in training)
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(
            X, labels, test_size=0.2, random_state=42, stratify=labels
        )

        print(f"Test data shape: {X_test.shape}")
        print(f"Test labels shape: {y_test.shape}")

    except Exception as e:
        print(f"Error loading data: {e}")
        return

    # Check if required files exist
    if not os.path.exists(BEST_MODEL_PATH):
        print(f"Model file not found: {BEST_MODEL_PATH}")
        return

    if not os.path.exists(HISTORY_PATH):
        print(f"History file not found: {HISTORY_PATH}")
        return

    # Generate plots
    print("Generating metrics plots...")
    plot_metrics_row(
        model_path=BEST_MODEL_PATH,
        X_test=X_test,
        y_test=y_test,
        history_path=HISTORY_PATH,
        save_path=PLOT_SAVE_PATH
    )

    print("Done!")

if __name__ == "__main__":
    main()


Loading test data...
Test data shape: (88, 30, 188)
Test labels shape: (88,)
Generating metrics plots...
Error loading model: <class 'keras.src.models.functional.Functional'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'keras.src.models.functional', 'class_name': 'Functional', 'config': {}, 'registered_name': 'Functional', 'build_config': {'input_shape': None}, 'compile_config': {'optimizer': {'module': 'keras.optimizers', 'class_name': 'Adam', 'config': {'name': 'adam', 'learning_rate': 0.0010000000474974513, 'weight_decay': None, 'clipnorm': None, 'global_clipnorm': None, 'clipvalue': None, 'use_ema': False, 'ema_momentum': 0.99, 'ema_overwrite_frequency': None, 'loss_scale_factor': None, 'gradient_accumulation_steps': None, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}, 're